# 支持向量机SVM
有人认为SVM是最好的现成的分类器，“现成”指的是分类器不加修改即可直接使用。

## 基于最大间隔分隔数据
优点：泛化错误率低，计算开销不大，结果易解释   
缺点：对参数调节和核函数的选择敏感，原始分类器不加修改仅适用于处理二类问题  
适用数据类型：数值型和标称型数据  
  
**将数据分隔开来的之间称为分隔超平面**  
**点到分隔面的距离称为间隔**  
**支持向量就是离分隔超平面最近的那些点**

## 寻找最大间隔

## SVM 应用的一般框架
1. 收集数据：可以使用任意方法
2. 准备数据：需要数值型数据
3. 分析数据：有助于可视化分隔超平面
4. 训练算法：SVM的大部分时间都源自训练，该过程主要实现两个参数的调优
5. 测试算法：十分简单的计算过程就可以实现
6. 使用算法：几乎所有分类问题都可以使用SVM,值得一提的是，SVM本身是一个二类分类器，对多类问题应用SVM需要对代码做一些修改

## SMO高效优化算法

In [12]:
## SMO算法中的辅助函数
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

# i是第一个alpha的下标，m是所有alpha的数目，只要函数值不等于输入值i，函数就会进行随机选择
def selectJrand(i,m):
    j = i
    while j==i:
        j = int(random.uniform(0,m))
    return j

# 用于调整大于H或者小于L的alpha值
def clipAlpha(aj,H,L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

In [11]:
dataArr,labelArr = loadDataSet('testSet.txt')
# labelArr

In [12]:
## 简化版SMO算法
# 5个输入参数，数据集，类别标签，常熟C，容错率和腿粗前最大的循环次数
def smoSimple(dataMatIn,classLabels,C,toler,maxIter):
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()
    b = 0
    # m=100
    m,n = shape(dataMatrix)
    alphas = mat(zeros((m,1)))
    # iter变量用于存储没有任何alpha改变的情况下遍历数据集的次数//.
    iter = 0
    while iter < maxIter:
        # 该变量记录alpha是否已经进行优化
        alphaPairsChanged = 0
        for i in range(m):
            # multiply函数为对应元素相乘
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
            # print(fXi)
            Ei = fXi - float(labelMat[i])
            # 如果不满足KKT条件
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b
                Ej = fXj - float(labelMat[j])
                # 记录alphai和alphaj的原始值，便于后续的比较
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()
                if labelMat[i] != labelMat[j]:
                    L = max(0,alphas[j] - alphas[i])
                    H = min(C,C + alphas[j] - alphas[i])
                else:
                    L = max(0,alphas[j] + alphas[i] - C)
                    H = min(C,alphas[j] + alphas[i])
                if L == H:
                    print("L==H")
                    continue
                # eta是alpha[j]的最优修改量
                eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T - \
                    dataMatrix[i,:]*dataMatrix[i,:].T - dataMatrix[j,:]*dataMatrix[j,:].T
                if eta >= 0:
                    print("eta>=0")
                    continue
                alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                alphas[j] = clipAlpha(alphas[j],H,L)
                if abs(alphas[j] - alphaJold) < 0.00001:
                    print("j not moving enough")
                    continue
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])
                b1 = b - Ei - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - \
                    labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2 = b - Ej - labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - \
                    labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print("iter:%d i:%d,pairs changed %d" % (iter,i,alphaPairsChanged))
        if alphaPairsChanged ==0:
            iter += 1
        else:
            iter = 0
        print("iteration number:%d" % iter)
    return b,alphas

In [13]:
b,alphas = smoSimple(dataArr,labelArr,0.6,0.001,40)
# b
alphas[alphas>0]

L==H
iter:0 i:1,pairs changed 1
iter:0 i:3,pairs changed 2
L==H
L==H
L==H
L==H
L==H
iter:0 i:10,pairs changed 3
iter:0 i:17,pairs changed 4
L==H
j not moving enough
iter:0 i:22,pairs changed 5
iter:0 i:23,pairs changed 6
L==H
iter:0 i:39,pairs changed 7
j not moving enough
j not moving enough
L==H
iter:0 i:55,pairs changed 8
iter:0 i:81,pairs changed 9
iteration number:0
j not moving enough
j not moving enough
iter:0 i:4,pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iter:0 i:39,pairs changed 2
j not moving enough
j not moving enough
L==H
j not moving enough
iter:0 i:57,pairs changed 3
L==H
iteration number:0
j not moving enough
iter:0 i:3,pairs changed 1
L==H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L==H
j not moving enough
j not moving enough
L==H
L==H
j not moving enough
L==H
it

iteration number:2
j not moving enough
L==H
j not moving enough
j not moving enough
j not moving enough
iteration number:3
iter:3 i:17,pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:2
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:3
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:4
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:5
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:6
j not moving eno

j not moving enough
j not moving enough
iteration number:2
j not moving enough
j not moving enough
j not moving enough
iteration number:3
j not moving enough
j not moving enough
iter:3 i:55,pairs changed 1
iteration number:0
j not moving enough
j not moving enough
j not moving enough
iteration number:1
j not moving enough
j not moving enough
j not moving enough
iteration number:2
j not moving enough
j not moving enough
j not moving enough
iteration number:3
j not moving enough
j not moving enough
j not moving enough
iteration number:4
j not moving enough
j not moving enough
j not moving enough
iteration number:5
j not moving enough
j not moving enough
j not moving enough
iteration number:6
j not moving enough
j not moving enough
j not moving enough
iteration number:7
j not moving enough
j not moving enough
j not moving enough
iteration number:8
j not moving enough
j not moving enough
j not moving enough
iteration number:9
j not moving enough
j not moving enough
j not moving enough
iter

j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:16
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:17
j not moving enough
j not moving enough
iter:17 i:17,pairs changed 1
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:0
j not moving enough
j not moving

j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:15
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:16
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:17
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:18
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
iteration number:19
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough


matrix([[1.27474091e-01, 2.08166817e-17, 2.40920335e-01, 2.08166817e-17,
         2.08166817e-17, 3.68392207e-01, 6.24500451e-17, 1.56125113e-16]])

In [14]:
shape(alphas[alphas>0])

(1, 8)

## 利用完整Platt SMO算法加速优化

In [15]:
## 完整版Platt SMO的支持函数
class optStruct:
    def __init__(self,dataMatIn,classLabels,C,toler):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))

# 计算E值并返回
def calcEk(oS,k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T * (oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek

# 用于选择第二个alpha值以保证每次优化中采用最大步长
def selectJ(i,oS,Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    #将误差矩阵每一行第一列置1，以此确定出误差不为0的样本
    oS.eCache[i] = [1,Ei]
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if len(validEcacheList) > 1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS,k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK,Ej
    # 如果这是第一次循环，那么就随机选择一个alpha值
    else:
        j = selectJrand(i,oS.m)
        Ej = calcEk(oS,j)
    return j,Ej

def updateEk(oS,k):
    Ek = calcEk(oS,k)
    oS.eCache[k] = [1,Ek]

In [16]:
## 完整platt SMO算法中的优化例程
def innerL(i,oS):
    Ei = calcEk(oS,i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or \
        ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i]) > 0):
        j,Ej = selectJ(i,oS,Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0,oS.alphas[j] - oS.alphas[i])
            H = min(oS.C,oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0,oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C,oS.alphas[j] + oS.alphas[i])
        if L == H:
            print('L==H')
            return 0
        eta = 2.0 * oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - \
            oS.X[j,:]*oS.X[j,:].T
        if eta >=0:
            print('eta>=0')
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        if abs(oS.alphas[j] - alphaJold) < 0.00001:
            print('j not moving enough')
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS,i)
        b1 = oS.b - Ei - oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[i,:].T - \
            oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
        b2 = oS.b - Ej - oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[j,:].T - \
            oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2
        return 1
    else:
        return 0

In [28]:
## 完整版Platt SMO的外循环代码
def smoP(dataMatIn,classLabels,C,toler,maxIter,kTup=('lin',0)):
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler,kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    #选取第一个变量alpha的三种情况，从间隔边界上选取或者整个数据集
    while (iter < maxIter) and ((alphaPairsChanged > 0) or entireSet):
        alphaPairsChanged = 0
        # 没有alpha更新对
        if entireSet:
            for i in range(oS.m):
                alphaPairsChanged += innerL(i,oS)
                print("fullSet,iter:%d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:
            #统计alphas向量中满足0<alpha<C的alpha列表
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
#             print(nonBoundIs)
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print("non-bound, iter: %d i: %d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet:
            entireSet = False
        elif alphaPairsChanged == 0:
            entireSet = True
        print("iteration number: %d" % iter)
    return oS.b,oS.alphas

In [18]:
dataArr,labelArr = loadDataSet('testSet.txt')
b,alphas = smoP(dataArr,labelArr,0.6,0.001,40)

fullSet,iter:0 i:0, pairs changed 1
fullSet,iter:0 i:1, pairs changed 1
fullSet,iter:0 i:2, pairs changed 1
fullSet,iter:0 i:3, pairs changed 1
fullSet,iter:0 i:4, pairs changed 1
fullSet,iter:0 i:5, pairs changed 1
fullSet,iter:0 i:6, pairs changed 1
fullSet,iter:0 i:7, pairs changed 1
fullSet,iter:0 i:8, pairs changed 2
fullSet,iter:0 i:9, pairs changed 2
fullSet,iter:0 i:10, pairs changed 3
fullSet,iter:0 i:11, pairs changed 3
fullSet,iter:0 i:12, pairs changed 3
fullSet,iter:0 i:13, pairs changed 3
fullSet,iter:0 i:14, pairs changed 3
fullSet,iter:0 i:15, pairs changed 3
fullSet,iter:0 i:16, pairs changed 3
L==H
fullSet,iter:0 i:17, pairs changed 3
fullSet,iter:0 i:18, pairs changed 3
fullSet,iter:0 i:19, pairs changed 3
fullSet,iter:0 i:20, pairs changed 3
fullSet,iter:0 i:21, pairs changed 3
fullSet,iter:0 i:22, pairs changed 3
L==H
fullSet,iter:0 i:23, pairs changed 3
fullSet,iter:0 i:24, pairs changed 3
fullSet,iter:0 i:25, pairs changed 3
fullSet,iter:0 i:26, pairs changed 3
f

In [23]:
## 计算w
def calcWs(alphas,dataArr,classLabels):
    X = mat(dataArr)
    labelMat = mat(classLabels).transpose()
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [24]:
ws = calcWs(alphas,dataArr,labelArr)
ws

array([[ 0.7341525 ],
       [-0.25148672]])

In [29]:
## 对数据进行分类处理，比如对第一个数据点分类
# 如果该值大于0，那么其属于1类；如果该值小于0，则属于-1类，对于数据点0，应该得到类别标签-1
dataMat = mat(dataArr)
dataMat[0]*mat(ws)+b

matrix([[-1.25528871]])

In [26]:
labelArr[0]

-1.0

In [27]:
dataMat[2]*mat(ws)+b

matrix([[2.5825929]])

In [28]:
labelArr[2]

1.0

## 在复杂数据上应用核函数
从某个特征空间到另一个特征空间的映射是通过核函数来实现的

In [22]:
## 核转换函数
##输入参数,X:支持向量的特征树；A：某一行特征数据；kTup：('lin',k1)核函数的类型和参数
def kernelTrans(X,A,kTup):
    m,n = shape(X)
    K = mat(zeros((m,1)))
    #线性函数
    if kTup[0] == 'lin':
        K = X * A.T
    # 径向基函数(radial bias function)
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow * deltaRow.T
        K = exp(K/(-1*kTup[1]**2))
    else:
        raise NameError('Houston We have a problem--That Kernel is not recognized')
    return K

class optStruct:
    def __init__(self,dataMatIn,classLabels,C,toler,kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        #软间隔参数C，参数越大，非线性拟合能力越强
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2)))
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X,self.X[i,:],kTup)

In [29]:
## 使用核函数时需要对innerL()和calcEk()函数进行修改
def innerL(i,oS):
    Ei = calcEk(oS,i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or \
        ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i]) > 0):
        j,Ej = selectJ(i,oS,Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()
        if oS.labelMat[i] != oS.labelMat[j]:
            L = max(0,oS.alphas[j] - oS.alphas[i])
            H = min(oS.C,oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0,oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C,oS.alphas[j] + oS.alphas[i])
        if L == H:
            print('L==H')
            return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]
        if eta >=0:
            print('eta>=0')
            return 0
        oS.alphas[j] -= oS.labelMat[j] * (Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        if abs(oS.alphas[j] - alphaJold) < 0.00001:
            print('j not moving enough')
        oS.alphas[i] += oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        updateEk(oS,i)
        b1 = oS.b - Ei - oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - \
            oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej - oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j] - \
            oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2)/2
        return 1
    else:
        return 0

# 计算E值并返回
def calcEk(oS,k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T * oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek

# 用于选择第二个alpha值以保证每次优化中采用最大步长
def selectJ(i,oS,Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    #将误差矩阵每一行第一列置1，以此确定出误差不为0的样本
    oS.eCache[i] = [1,Ei]
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if len(validEcacheList) > 1:
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(oS,k)
            deltaE = abs(Ei - Ek)
            if deltaE > maxDeltaE:
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK,Ej
    # 如果这是第一次循环，那么就随机选择一个alpha值
    else:
        j = selectJrand(i,oS.m)
        Ej = calcEk(oS,j)
    return j,Ej

def updateEk(oS,k):
    Ek = calcEk(oS,k)
    oS.eCache[k] = [1,Ek]

## 在测试中使用核函数

In [70]:
## 利用核函数进行分类的径向基测试函数
def testRbf(k1=1.3):
    dataArr,labelArr = loadDataSet('testSetRBF.txt')
    b,alphas = smoP(dataArr,labelArr,200,0.0001,10,('rbf',k1))
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    #选取不为0数据的行数（也就是支持向量）
    svInd = nonzero(alphas.A > 0)[0]
    ## 数组索引数组，得到支持向量
    sVs = dataMat[svInd]
    #支持向量的类别标签（1或-1）
    labelSV = labelMat[svInd]
    print("there are %d support Vectors" % shape(sVs)[0])
    m,n = shape(dataMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],('rbf',k1))
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        # sign函数是一个符号函数，x>0,sign(x)=1;x=0,sign(x)=0;x<0,sign(x)=-1
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    m,n = shape(dataMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],('rbf',k1))
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("the test error rate is %f" % (errorCount/m))

In [71]:
testRbf()

fullSet,iter:0 i:0, pairs changed 1
fullSet,iter:0 i:1, pairs changed 1
fullSet,iter:0 i:2, pairs changed 2
fullSet,iter:0 i:3, pairs changed 3
fullSet,iter:0 i:4, pairs changed 3
fullSet,iter:0 i:5, pairs changed 4
fullSet,iter:0 i:6, pairs changed 5
fullSet,iter:0 i:7, pairs changed 5
fullSet,iter:0 i:8, pairs changed 6
fullSet,iter:0 i:9, pairs changed 6
fullSet,iter:0 i:10, pairs changed 7
fullSet,iter:0 i:11, pairs changed 7
fullSet,iter:0 i:12, pairs changed 7
fullSet,iter:0 i:13, pairs changed 8
fullSet,iter:0 i:14, pairs changed 9
fullSet,iter:0 i:15, pairs changed 10
fullSet,iter:0 i:16, pairs changed 11
fullSet,iter:0 i:17, pairs changed 12
fullSet,iter:0 i:18, pairs changed 12
fullSet,iter:0 i:19, pairs changed 12
fullSet,iter:0 i:20, pairs changed 12
L==H
fullSet,iter:0 i:21, pairs changed 12
fullSet,iter:0 i:22, pairs changed 12
fullSet,iter:0 i:23, pairs changed 13
fullSet,iter:0 i:24, pairs changed 13
fullSet,iter:0 i:25, pairs changed 13
fullSet,iter:0 i:26, pairs chang

non-bound, iter: 4 i: 53, pairs changed 19
j not moving enough
non-bound, iter: 4 i: 54, pairs changed 20
j not moving enough
non-bound, iter: 4 i: 57, pairs changed 21
j not moving enough
non-bound, iter: 4 i: 58, pairs changed 22
j not moving enough
non-bound, iter: 4 i: 74, pairs changed 23
j not moving enough
non-bound, iter: 4 i: 80, pairs changed 24
j not moving enough
non-bound, iter: 4 i: 86, pairs changed 25
j not moving enough
non-bound, iter: 4 i: 87, pairs changed 26
iteration number: 5
j not moving enough
non-bound, iter: 5 i: 0, pairs changed 1
j not moving enough
non-bound, iter: 5 i: 2, pairs changed 2
j not moving enough
non-bound, iter: 5 i: 3, pairs changed 3
j not moving enough
non-bound, iter: 5 i: 6, pairs changed 4
j not moving enough
non-bound, iter: 5 i: 10, pairs changed 5
j not moving enough
non-bound, iter: 5 i: 13, pairs changed 6
j not moving enough
non-bound, iter: 5 i: 14, pairs changed 7
j not moving enough
non-bound, iter: 5 i: 16, pairs changed 8
j no

the training error rate is: 0.050000
the test error rate is 0.120000


## 手写识别问题
支持向量的数目存在一个最优值，如果支持向量太多，相当于每次都利用整个数据集进行分类，这种方法称为K近邻  
如果支持向量太少，可能会得到一个很差的决策边界

## 基于SVM的数字识别
1. 收集数据：提供的文本文件
2. 准备数据：基于二值图像构造向量
3. 分析数据： 对图像向量进行目测
4. 训练算法： 采用两种不同的核函数，并对径向基函数采用不同的设置来运行SMO算法
5. 测试算法： 编写一个函数来测试不同的核函数并计算错误率
6. 使用算法： 一个图像识别的完整应用还需要一些图像处理的知识

In [66]:
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect    

## 基于SVM的手写数字识别
def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9:
            hwLabels.append(-1)
        else:
            hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName,fileNameStr))
    return trainingMat,hwLabels

def testDigits(kTup=('rbf',10)):
    dataArr,labelArr = loadImages('../KNN/trainingDigits')
    b,alphas = smoP(dataArr,labelArr,200,0.001,40,kTup)
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A>0)[0]
    sVs = data[svInd]
    labelSV = labelMat[svInd]
    print("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(dataMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],kTup)
        predict = kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount +=1 
    print("the training error rate is %f" % (errorCount / m))
    dataArr,labelArr = loadImages('testDigits')
    errorCount = 0 
    dataMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    m,n = shape(dataMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,dataMat[i,:],kTup)
        predict = kernelEval.T  * multiply(labelSV,alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
    print("the test error rate is: %f" % (errorCount/m))

In [69]:
testDigits()

L==H
fullSet,iter:0 i:0, pairs changed 0
fullSet,iter:0 i:1, pairs changed 1
fullSet,iter:0 i:2, pairs changed 2
fullSet,iter:0 i:3, pairs changed 3
fullSet,iter:0 i:4, pairs changed 4
fullSet,iter:0 i:5, pairs changed 5
L==H
fullSet,iter:0 i:6, pairs changed 5
fullSet,iter:0 i:7, pairs changed 6
fullSet,iter:0 i:8, pairs changed 7
fullSet,iter:0 i:9, pairs changed 8
fullSet,iter:0 i:10, pairs changed 9
L==H
fullSet,iter:0 i:11, pairs changed 9
fullSet,iter:0 i:12, pairs changed 10
fullSet,iter:0 i:13, pairs changed 11
fullSet,iter:0 i:14, pairs changed 12
fullSet,iter:0 i:15, pairs changed 13
fullSet,iter:0 i:16, pairs changed 14
fullSet,iter:0 i:17, pairs changed 15
fullSet,iter:0 i:18, pairs changed 16
fullSet,iter:0 i:19, pairs changed 16
fullSet,iter:0 i:20, pairs changed 17
fullSet,iter:0 i:21, pairs changed 18
L==H
fullSet,iter:0 i:22, pairs changed 18
fullSet,iter:0 i:23, pairs changed 18
L==H
fullSet,iter:0 i:24, pairs changed 18
L==H
fullSet,iter:0 i:25, pairs changed 18
L==H

L==H
fullSet,iter:0 i:201, pairs changed 18
L==H
fullSet,iter:0 i:202, pairs changed 18
L==H
fullSet,iter:0 i:203, pairs changed 18
L==H
fullSet,iter:0 i:204, pairs changed 18
L==H
fullSet,iter:0 i:205, pairs changed 18
L==H
fullSet,iter:0 i:206, pairs changed 18
L==H
fullSet,iter:0 i:207, pairs changed 18
L==H
fullSet,iter:0 i:208, pairs changed 18
L==H
fullSet,iter:0 i:209, pairs changed 18
L==H
fullSet,iter:0 i:210, pairs changed 18
L==H
fullSet,iter:0 i:211, pairs changed 18
L==H
fullSet,iter:0 i:212, pairs changed 18
L==H
fullSet,iter:0 i:213, pairs changed 18
L==H
fullSet,iter:0 i:214, pairs changed 18
L==H
fullSet,iter:0 i:215, pairs changed 18
L==H
fullSet,iter:0 i:216, pairs changed 18
L==H
fullSet,iter:0 i:217, pairs changed 18
L==H
fullSet,iter:0 i:218, pairs changed 18
L==H
fullSet,iter:0 i:219, pairs changed 18
L==H
fullSet,iter:0 i:220, pairs changed 18
L==H
fullSet,iter:0 i:221, pairs changed 18
L==H
fullSet,iter:0 i:222, pairs changed 18
L==H
fullSet,iter:0 i:223, pairs

L==H
fullSet,iter:0 i:388, pairs changed 18
L==H
fullSet,iter:0 i:389, pairs changed 18
L==H
fullSet,iter:0 i:390, pairs changed 18
L==H
fullSet,iter:0 i:391, pairs changed 18
L==H
fullSet,iter:0 i:392, pairs changed 18
L==H
fullSet,iter:0 i:393, pairs changed 18
L==H
fullSet,iter:0 i:394, pairs changed 18
L==H
fullSet,iter:0 i:395, pairs changed 18
L==H
fullSet,iter:0 i:396, pairs changed 18
L==H
fullSet,iter:0 i:397, pairs changed 18
L==H
fullSet,iter:0 i:398, pairs changed 18
L==H
fullSet,iter:0 i:399, pairs changed 18
L==H
fullSet,iter:0 i:400, pairs changed 18
L==H
fullSet,iter:0 i:401, pairs changed 18
L==H
fullSet,iter:0 i:402, pairs changed 18
L==H
fullSet,iter:0 i:403, pairs changed 18
L==H
fullSet,iter:0 i:404, pairs changed 18
L==H
fullSet,iter:0 i:405, pairs changed 18
L==H
fullSet,iter:0 i:406, pairs changed 18
L==H
fullSet,iter:0 i:407, pairs changed 18
L==H
fullSet,iter:0 i:408, pairs changed 18
L==H
fullSet,iter:0 i:409, pairs changed 18
L==H
fullSet,iter:0 i:410, pairs

L==H
fullSet,iter:0 i:575, pairs changed 18
L==H
fullSet,iter:0 i:576, pairs changed 18
L==H
fullSet,iter:0 i:577, pairs changed 18
L==H
fullSet,iter:0 i:578, pairs changed 18
L==H
fullSet,iter:0 i:579, pairs changed 18
L==H
fullSet,iter:0 i:580, pairs changed 18
L==H
fullSet,iter:0 i:581, pairs changed 18
L==H
fullSet,iter:0 i:582, pairs changed 18
L==H
fullSet,iter:0 i:583, pairs changed 18
L==H
fullSet,iter:0 i:584, pairs changed 18
L==H
fullSet,iter:0 i:585, pairs changed 18
L==H
fullSet,iter:0 i:586, pairs changed 18
L==H
fullSet,iter:0 i:587, pairs changed 18
L==H
fullSet,iter:0 i:588, pairs changed 18
L==H
fullSet,iter:0 i:589, pairs changed 18
L==H
fullSet,iter:0 i:590, pairs changed 18
L==H
fullSet,iter:0 i:591, pairs changed 18
L==H
fullSet,iter:0 i:592, pairs changed 18
L==H
fullSet,iter:0 i:593, pairs changed 18
L==H
fullSet,iter:0 i:594, pairs changed 18
L==H
fullSet,iter:0 i:595, pairs changed 18
L==H
fullSet,iter:0 i:596, pairs changed 18
L==H
fullSet,iter:0 i:597, pairs

L==H
fullSet,iter:0 i:764, pairs changed 18
L==H
fullSet,iter:0 i:765, pairs changed 18
L==H
fullSet,iter:0 i:766, pairs changed 18
L==H
fullSet,iter:0 i:767, pairs changed 18
L==H
fullSet,iter:0 i:768, pairs changed 18
L==H
fullSet,iter:0 i:769, pairs changed 18
L==H
fullSet,iter:0 i:770, pairs changed 18
L==H
fullSet,iter:0 i:771, pairs changed 18
L==H
fullSet,iter:0 i:772, pairs changed 18
L==H
fullSet,iter:0 i:773, pairs changed 18
L==H
fullSet,iter:0 i:774, pairs changed 18
L==H
fullSet,iter:0 i:775, pairs changed 18
L==H
fullSet,iter:0 i:776, pairs changed 18
L==H
fullSet,iter:0 i:777, pairs changed 18
L==H
fullSet,iter:0 i:778, pairs changed 18
L==H
fullSet,iter:0 i:779, pairs changed 18
L==H
fullSet,iter:0 i:780, pairs changed 18
L==H
fullSet,iter:0 i:781, pairs changed 18
L==H
fullSet,iter:0 i:782, pairs changed 18
L==H
fullSet,iter:0 i:783, pairs changed 18
L==H
fullSet,iter:0 i:784, pairs changed 18
L==H
fullSet,iter:0 i:785, pairs changed 18
L==H
fullSet,iter:0 i:786, pairs

...

L==H
fullSet,iter:0 i:1503, pairs changed 18
L==H
fullSet,iter:0 i:1504, pairs changed 18
L==H
fullSet,iter:0 i:1505, pairs changed 18
L==H
fullSet,iter:0 i:1506, pairs changed 18
L==H
fullSet,iter:0 i:1507, pairs changed 18
L==H
fullSet,iter:0 i:1508, pairs changed 18
L==H
fullSet,iter:0 i:1509, pairs changed 18
L==H
fullSet,iter:0 i:1510, pairs changed 18
L==H
fullSet,iter:0 i:1511, pairs changed 18
L==H
fullSet,iter:0 i:1512, pairs changed 18
L==H
fullSet,iter:0 i:1513, pairs changed 18
L==H
fullSet,iter:0 i:1514, pairs changed 18
L==H
fullSet,iter:0 i:1515, pairs changed 18
L==H
fullSet,iter:0 i:1516, pairs changed 18
L==H
fullSet,iter:0 i:1517, pairs changed 18
L==H
fullSet,iter:0 i:1518, pairs changed 18
L==H
fullSet,iter:0 i:1519, pairs changed 18
L==H
fullSet,iter:0 i:1520, pairs changed 18
L==H
fullSet,iter:0 i:1521, pairs changed 18
L==H
fullSet,iter:0 i:1522, pairs changed 18
L==H
fullSet,iter:0 i:1523, pairs changed 18
L==H
fullSet,iter:0 i:1524, pairs changed 18
L==H
fullS

L==H
fullSet,iter:0 i:1687, pairs changed 18
L==H
fullSet,iter:0 i:1688, pairs changed 18
L==H
fullSet,iter:0 i:1689, pairs changed 18
L==H
fullSet,iter:0 i:1690, pairs changed 18
L==H
fullSet,iter:0 i:1691, pairs changed 18
L==H
fullSet,iter:0 i:1692, pairs changed 18
L==H
fullSet,iter:0 i:1693, pairs changed 18
L==H
fullSet,iter:0 i:1694, pairs changed 18
L==H
fullSet,iter:0 i:1695, pairs changed 18
L==H
fullSet,iter:0 i:1696, pairs changed 18
L==H
fullSet,iter:0 i:1697, pairs changed 18
L==H
fullSet,iter:0 i:1698, pairs changed 18
L==H
fullSet,iter:0 i:1699, pairs changed 18
L==H
fullSet,iter:0 i:1700, pairs changed 18
L==H
fullSet,iter:0 i:1701, pairs changed 18
L==H
fullSet,iter:0 i:1702, pairs changed 18
L==H
fullSet,iter:0 i:1703, pairs changed 18
L==H
fullSet,iter:0 i:1704, pairs changed 18
L==H
fullSet,iter:0 i:1705, pairs changed 18
L==H
fullSet,iter:0 i:1706, pairs changed 18
L==H
fullSet,iter:0 i:1707, pairs changed 18
L==H
fullSet,iter:0 i:1708, pairs changed 18
L==H
fullS

fullSet,iter:0 i:1882, pairs changed 149
j not moving enough
fullSet,iter:0 i:1883, pairs changed 150
fullSet,iter:0 i:1884, pairs changed 150
fullSet,iter:0 i:1885, pairs changed 151
fullSet,iter:0 i:1886, pairs changed 152
j not moving enough
fullSet,iter:0 i:1887, pairs changed 153
fullSet,iter:0 i:1888, pairs changed 153
fullSet,iter:0 i:1889, pairs changed 153
fullSet,iter:0 i:1890, pairs changed 153
fullSet,iter:0 i:1891, pairs changed 153
fullSet,iter:0 i:1892, pairs changed 153
fullSet,iter:0 i:1893, pairs changed 153
fullSet,iter:0 i:1894, pairs changed 154
fullSet,iter:0 i:1895, pairs changed 155
fullSet,iter:0 i:1896, pairs changed 156
fullSet,iter:0 i:1897, pairs changed 156
fullSet,iter:0 i:1898, pairs changed 157
fullSet,iter:0 i:1899, pairs changed 158
j not moving enough
fullSet,iter:0 i:1900, pairs changed 159
fullSet,iter:0 i:1901, pairs changed 159
fullSet,iter:0 i:1902, pairs changed 160
fullSet,iter:0 i:1903, pairs changed 160
fullSet,iter:0 i:1904, pairs changed 1

non-bound, iter: 1 i: 1551, pairs changed 105
non-bound, iter: 1 i: 1565, pairs changed 106
non-bound, iter: 1 i: 1577, pairs changed 107


KeyboardInterrupt: 

## cookie:

## 1. nonzero函数是numpy中用于得到数组array中非零元素的位置（数组索引）的函数  
它的返回值是一个长度为a.ndim(数组a的轴数)的元组，元组的每个元素都是一个整数数组，其值为非零元素的下标在对应轴上的值。

1. 只有a中非零元素才会有索引值，那些零值元素没有索引值；

2. 返回的索引值数组是一个2维tuple数组，该tuple数组中包含一维的array数组。其中，一维array向量的个数与a的维数是一致的。

3. 索引值数组的每一个array均是从一个维度上来描述其索引值。比如，如果a是一个二维数组，则索引值数组有两个array，第一个array从行维度来描述索引值；第二个array从列维度来描述索引值。

4. transpose(np.nonzero(x))函数能够描述出每一个非零元素在不同维度的索引值。

5. 通过a[nonzero(a)]得到所有a中的非零值


In [25]:
oS = optStruct(mat(dataArr),mat(labelArr).transpose(),0.6,0.001)
nonzero((oS.alphas.A > 0) * (oS.alphas.A < 0.6))
nonzero(oS.alphas[:,0].A)
# oS.eCache[:,0].A

TypeError: __init__() missing 1 required positional argument: 'kTup'

## 2. 数组索引数组

In [49]:
x = arange(10,1,-1)
x

array([10,  9,  8,  7,  6,  5,  4,  3,  2])

In [51]:
x[array([3, 3, 1, 8])]

array([7, 7, 9, 2])

In [26]:
from numpy import *
alphas = mat(zeros((100,1)))
labelMat = mat(labelArr).transpose()
# labelMat
dataMatrix = mat(dataArr)
# dataMatrix
b = 0
fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[1,:].T)) + b
fXi
dataMatrix*dataMatrix[1,:].T

matrix([[-0.04766421],
        [ 0.01223176],
        [-0.08459451],
        [-0.0257267 ],
        [-0.08470145],
        [ 0.09494702],
        [-0.00018384],
        [-0.01096419],
        [-0.08373849],
        [ 0.00442478],
        [ 0.02346097],
        [-0.08369674],
        [ 0.02129762],
        [ 0.01460454],
        [ 0.01207902],
        [ 0.03226163],
        [ 0.00601086],
        [-0.02750356],
        [ 0.03569684],
        [-0.01528212],
        [-0.08634249],
        [ 0.061067  ],
        [-0.00854541],
        [-0.0189196 ],
        [-0.08095172],
        [-0.0778407 ],
        [ 0.03116402],
        [-0.03001386],
        [-0.00992438],
        [-0.03207938],
        [ 0.0012422 ],
        [-0.03820128],
        [-0.1016634 ],
        [-0.02264616],
        [ 0.01066873],
        [ 0.00311592],
        [-0.01874952],
        [-0.089366  ],
        [-0.00387977],
        [ 0.00828807],
        [ 0.00238088],
        [-0.0475246 ],
        [-0.05462491],
        [ 0

In [2]:
from numpy import *
testMat = mat(eye(4))
testMat[0][0]
nonzero(testMat[:,1] <= 0.5)

(array([0, 2, 3]), array([0, 0, 0]))

In [33]:
dataArr,labelArr = loadDataSet('testSetRBF.txt')
b,alphas = smoP(dataArr,labelArr,200,0.0001,40,('rbf',1.3))
nonzero(alphas.A>0)

L==H
fullSet,iter:0 i:0, pairs changed 0
fullSet,iter:0 i:1, pairs changed 1
fullSet,iter:0 i:2, pairs changed 2
fullSet,iter:0 i:3, pairs changed 3
fullSet,iter:0 i:4, pairs changed 3
fullSet,iter:0 i:5, pairs changed 4
fullSet,iter:0 i:6, pairs changed 4
fullSet,iter:0 i:7, pairs changed 5
fullSet,iter:0 i:8, pairs changed 5
fullSet,iter:0 i:9, pairs changed 5
fullSet,iter:0 i:10, pairs changed 6
fullSet,iter:0 i:11, pairs changed 7
fullSet,iter:0 i:12, pairs changed 7
fullSet,iter:0 i:13, pairs changed 8
fullSet,iter:0 i:14, pairs changed 9
fullSet,iter:0 i:15, pairs changed 10
fullSet,iter:0 i:16, pairs changed 11
fullSet,iter:0 i:17, pairs changed 12
fullSet,iter:0 i:18, pairs changed 13
fullSet,iter:0 i:19, pairs changed 14
fullSet,iter:0 i:20, pairs changed 14
fullSet,iter:0 i:21, pairs changed 15
j not moving enough
fullSet,iter:0 i:22, pairs changed 16
fullSet,iter:0 i:23, pairs changed 16
fullSet,iter:0 i:24, pairs changed 17
fullSet,iter:0 i:25, pairs changed 17
fullSet,iter

j not moving enough
non-bound, iter: 4 i: 19, pairs changed 10
j not moving enough
non-bound, iter: 4 i: 21, pairs changed 11
j not moving enough
non-bound, iter: 4 i: 26, pairs changed 12
j not moving enough
non-bound, iter: 4 i: 27, pairs changed 13
j not moving enough
non-bound, iter: 4 i: 28, pairs changed 14
j not moving enough
non-bound, iter: 4 i: 29, pairs changed 15
j not moving enough
non-bound, iter: 4 i: 31, pairs changed 16
j not moving enough
non-bound, iter: 4 i: 36, pairs changed 17
j not moving enough
non-bound, iter: 4 i: 41, pairs changed 18
j not moving enough
non-bound, iter: 4 i: 42, pairs changed 19
j not moving enough
non-bound, iter: 4 i: 45, pairs changed 20
j not moving enough
non-bound, iter: 4 i: 50, pairs changed 21
j not moving enough
non-bound, iter: 4 i: 58, pairs changed 22
j not moving enough
non-bound, iter: 4 i: 62, pairs changed 23
j not moving enough
non-bound, iter: 4 i: 65, pairs changed 24
j not moving enough
non-bound, iter: 4 i: 74, pairs cha

j not moving enough
non-bound, iter: 10 i: 74, pairs changed 25
j not moving enough
non-bound, iter: 10 i: 87, pairs changed 26
iteration number: 11
j not moving enough
non-bound, iter: 11 i: 0, pairs changed 1
j not moving enough
non-bound, iter: 11 i: 1, pairs changed 2
j not moving enough
non-bound, iter: 11 i: 3, pairs changed 3
j not moving enough
non-bound, iter: 11 i: 10, pairs changed 4
j not moving enough
non-bound, iter: 11 i: 13, pairs changed 5
j not moving enough
non-bound, iter: 11 i: 14, pairs changed 6
j not moving enough
non-bound, iter: 11 i: 16, pairs changed 7
j not moving enough
non-bound, iter: 11 i: 17, pairs changed 8
j not moving enough
non-bound, iter: 11 i: 18, pairs changed 9
j not moving enough
non-bound, iter: 11 i: 19, pairs changed 10
j not moving enough
non-bound, iter: 11 i: 21, pairs changed 11
j not moving enough
non-bound, iter: 11 i: 26, pairs changed 12
j not moving enough
non-bound, iter: 11 i: 27, pairs changed 13
j not moving enough
non-bound, 

j not moving enough
non-bound, iter: 17 i: 58, pairs changed 22
j not moving enough
non-bound, iter: 17 i: 62, pairs changed 23
j not moving enough
non-bound, iter: 17 i: 65, pairs changed 24
j not moving enough
non-bound, iter: 17 i: 74, pairs changed 25
j not moving enough
non-bound, iter: 17 i: 87, pairs changed 26
iteration number: 18
j not moving enough
non-bound, iter: 18 i: 0, pairs changed 1
j not moving enough
non-bound, iter: 18 i: 1, pairs changed 2
j not moving enough
non-bound, iter: 18 i: 3, pairs changed 3
j not moving enough
non-bound, iter: 18 i: 10, pairs changed 4
j not moving enough
non-bound, iter: 18 i: 13, pairs changed 5
j not moving enough
non-bound, iter: 18 i: 14, pairs changed 6
j not moving enough
non-bound, iter: 18 i: 16, pairs changed 7
j not moving enough
non-bound, iter: 18 i: 17, pairs changed 8
j not moving enough
non-bound, iter: 18 i: 18, pairs changed 9
j not moving enough
non-bound, iter: 18 i: 19, pairs changed 10
j not moving enough
non-bound, 

j not moving enough
non-bound, iter: 23 i: 3, pairs changed 3
j not moving enough
non-bound, iter: 23 i: 10, pairs changed 4
j not moving enough
non-bound, iter: 23 i: 13, pairs changed 5
j not moving enough
non-bound, iter: 23 i: 14, pairs changed 6
j not moving enough
non-bound, iter: 23 i: 16, pairs changed 7
j not moving enough
non-bound, iter: 23 i: 17, pairs changed 8
j not moving enough
non-bound, iter: 23 i: 18, pairs changed 9
j not moving enough
non-bound, iter: 23 i: 19, pairs changed 10
j not moving enough
non-bound, iter: 23 i: 21, pairs changed 11
j not moving enough
non-bound, iter: 23 i: 26, pairs changed 12
j not moving enough
non-bound, iter: 23 i: 27, pairs changed 13
j not moving enough
non-bound, iter: 23 i: 28, pairs changed 14
j not moving enough
non-bound, iter: 23 i: 29, pairs changed 15
j not moving enough
non-bound, iter: 23 i: 31, pairs changed 16
j not moving enough
non-bound, iter: 23 i: 36, pairs changed 17
j not moving enough
non-bound, iter: 23 i: 41, p

j not moving enough
non-bound, iter: 28 i: 42, pairs changed 19
j not moving enough
non-bound, iter: 28 i: 45, pairs changed 20
j not moving enough
non-bound, iter: 28 i: 50, pairs changed 21
j not moving enough
non-bound, iter: 28 i: 58, pairs changed 22
j not moving enough
non-bound, iter: 28 i: 62, pairs changed 23
j not moving enough
non-bound, iter: 28 i: 65, pairs changed 24
j not moving enough
non-bound, iter: 28 i: 74, pairs changed 25
j not moving enough
non-bound, iter: 28 i: 87, pairs changed 26
iteration number: 29
j not moving enough
non-bound, iter: 29 i: 0, pairs changed 1
j not moving enough
non-bound, iter: 29 i: 1, pairs changed 2
j not moving enough
non-bound, iter: 29 i: 3, pairs changed 3
j not moving enough
non-bound, iter: 29 i: 10, pairs changed 4
j not moving enough
non-bound, iter: 29 i: 13, pairs changed 5
j not moving enough
non-bound, iter: 29 i: 14, pairs changed 6
j not moving enough
non-bound, iter: 29 i: 16, pairs changed 7
j not moving enough
non-bound

j not moving enough
non-bound, iter: 34 i: 74, pairs changed 25
j not moving enough
non-bound, iter: 34 i: 87, pairs changed 26
iteration number: 35
j not moving enough
non-bound, iter: 35 i: 0, pairs changed 1
j not moving enough
non-bound, iter: 35 i: 1, pairs changed 2
j not moving enough
non-bound, iter: 35 i: 3, pairs changed 3
j not moving enough
non-bound, iter: 35 i: 10, pairs changed 4
j not moving enough
non-bound, iter: 35 i: 13, pairs changed 5
j not moving enough
non-bound, iter: 35 i: 14, pairs changed 6
j not moving enough
non-bound, iter: 35 i: 16, pairs changed 7
j not moving enough
non-bound, iter: 35 i: 17, pairs changed 8
j not moving enough
non-bound, iter: 35 i: 18, pairs changed 9
j not moving enough
non-bound, iter: 35 i: 19, pairs changed 10
j not moving enough
non-bound, iter: 35 i: 21, pairs changed 11
j not moving enough
non-bound, iter: 35 i: 26, pairs changed 12
j not moving enough
non-bound, iter: 35 i: 27, pairs changed 13
j not moving enough
non-bound, 

(array([ 0,  1,  3, 10, 13, 14, 16, 17, 18, 19, 21, 26, 27, 28, 29, 31, 36,
        41, 42, 45, 50, 58, 62, 65, 74, 87]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0]))

In [55]:
svInd = nonzero(alphas.A>0)[0]
dataMat = mat(dataArr)
sVs = dataMat[svInd]
shape(sVs)
# dataMat

matrix([[-2.14824e-01,  6.62756e-01],
        [-6.15690e-02, -9.18750e-02],
        [ 2.23650e-01,  1.30142e-01],
        [-1.52696e-01, -1.53030e-01],
        [-2.38076e-01,  5.83000e-04],
        [-4.13576e-01,  1.45681e-01],
        [ 1.99894e-01, -1.99381e-01],
        [-3.56048e-01,  5.37960e-01],
        [-3.92868e-01, -1.25261e-01],
        [ 3.53588e-01, -7.06170e-02],
        [-4.75167e-01, -3.46247e-01],
        [-4.49349e-01, -3.80740e-02],
        [ 6.19080e-01, -8.81880e-02],
        [ 2.68066e-01, -7.16210e-02],
        [-1.51650e-02,  3.59326e-01],
        [-3.19153e-01,  6.29673e-01],
        [-4.03483e-01,  4.74466e-01],
        [-1.29980e-02,  5.25985e-01],
        [ 1.53738e-01,  4.91531e-01],
        [ 6.82860e-02,  3.92741e-01],
        [ 3.53980e-02, -3.33682e-01],
        [-3.07768e-01,  5.03038e-01],
        [-2.03272e-01,  2.86855e-01],
        [-7.55431e-01,  9.67110e-02],
        [-4.48939e-01,  1.76725e-01],
        [ 5.41359e-01, -2.05969e-01]])

In [46]:
dataMat[array([0,1,3])]

matrix([[-0.214824,  0.662756],
        [-0.061569, -0.091875],
        [ 0.22365 ,  0.130142]])

In [41]:
len(dataMat[svInd])

26

In [65]:
pwd

'/Users/kokenhei/Desktop/study/知识点整理/支持向量机'